In [56]:
// << Ammonite Downloads >>
// Log4j
import $ivy.`org.apache.logging.log4j:log4j-core:2.13.3`
// Spark
import $ivy.`org.apache.spark::spark-core:3.0.0` // Or use any other 3.x version here
import $ivy.`org.apache.spark::spark-sql:3.0.0` // Or use any other 3.x version here
import $ivy.`org.plotly-scala::plotly-almond:0.7.2`

import $ivy.$                                           
// Spark

import $ivy.$                                    // Or use any other 3.x version here

import $ivy.$                                   // Or use any other 3.x version here

import $ivy.$                                      

In [57]:
// << Init imports >>
import org.apache.log4j.{Logger,Level}
import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

import org.apache.log4j.{Logger,Level}

import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._

In [58]:
// << Logger setting >>
Logger.getLogger("org").setLevel(Level.WARN)
Logger.getLogger("akka").setLevel(Level.WARN)

In [59]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Getting spark JARs
Creating SparkSession


20/07/11 00:22:18 WARN NotebookSparkSessionBuilder: Using an existing SparkSession; some spark core configurations may not take effect.


Spark UI

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@375dc845

In [60]:
// Exam
import spark.implicits._
val df = spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("../data/spark_the_definitive_guide/Spark-The-Definitive-Guide/data/retail-data/by-day/2010-12-01.csv")
df.createOrReplaceTempView("dfTable")

load at cmd59.sc:5

1 / 1

load at cmd59.sc:5

1 / 1

load at cmd59.sc:5

...

load at cmd59.sc:5

2 / 2

load at cmd59.sc:5

2 / 2

import spark.implicits._

df: DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]

In [61]:
spark.sql("""
    SELECT ltrim('   HELLO  ') AS ltrim, lpad('HELLLLLLLL', 3, ' ') AS lpad
    FROM dfTable
""").show(2)

show at cmd60.sc:4

1 / 1

show at cmd60.sc:4

1 / 1

show at cmd60.sc:4

1 / 1

+-------+----+
|  ltrim|lpad|
+-------+----+
|HELLO  | HEL|
|HELLO  | HEL|
+-------+----+
only showing top 2 rows



In [62]:
val simpleColors = Seq("black", "white", "red", "green", "blue")
val regexString = simpleColors.map(_.toUpperCase).mkString("|")
println(regexString)

df.select(
    regexp_replace($"Description", regexString, "COLOR").alias("color_cleam"),
    $"Description"
).show(2)

BLACK|WHITE|RED|GREEN|BLUE


show at cmd61.sc:8

1 / 1

show at cmd61.sc:8

1 / 1

show at cmd61.sc:8

1 / 1

+--------------------+--------------------+
|         color_cleam|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows



simpleColors: Seq[String] = List("black", "white", "red", "green", "blue")
regexString: String = "BLACK|WHITE|RED|GREEN|BLUE"

In [63]:
spark.sql("""
    SELECT regexp_replace(Description, 'BLACK|WHITE|RED|GREEN|BLUE', 'COLOR') AS color_clean, Description
    FROM dfTable
""").show(2)

show at cmd62.sc:4

1 / 1

show at cmd62.sc:4

1 / 1

show at cmd62.sc:4

1 / 1

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows



In [64]:
spark.sql("""
    SELECT regexp_extract(Description, '(BLACK|WHITE|RED|GREEN|BLUE)', 1) AS color_clean, Description
    FROM dfTable
""").show(3)

show at cmd63.sc:4

1 / 1

show at cmd63.sc:4

1 / 1

show at cmd63.sc:4

1 / 1

+-----------+--------------------+
|color_clean|         Description|
+-----------+--------------------+
|      WHITE|WHITE HANGING HEA...|
|      WHITE| WHITE METAL LANTERN|
|           |CREAM CUPID HEART...|
+-----------+--------------------+
only showing top 3 rows



In [65]:
val containsBlack = $"Description".contains("BLACK")
val containsWhite = $"Description".contains("WHITE")
df.withColumn("hasSimpleColor", containsBlack.or(containsWhite))
    .where("hasSimpleColor")
    .select("Description").show(3, false)

show at cmd64.sc:5

1 / 1

show at cmd64.sc:5

1 / 1

show at cmd64.sc:5

1 / 1

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



containsBlack: Column = contains(Description, BLACK)
containsWhite: Column = contains(Description, WHITE)

In [66]:
spark.sql("""
    SELECT Description
    FROM dfTable
    WHERE instr(Description, 'BLACK') >= 1 OR instr(Description, 'WHITE') >= 1
""").show(3, false)

show at cmd65.sc:5

1 / 1

show at cmd65.sc:5

1 / 1

show at cmd65.sc:5

1 / 1

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



In [67]:
// scala
val simpleColors = Seq("black", "white", "red", "green", "blue")
val selectedColumns = simpleColors.map(color => {
    $"Description".contains(color.toUpperCase).alias(s"is_$color")
}):+expr("*") // append * value
// _* varargs
df.select(selectedColumns:_*).where($"is_white".or($"is_red"))
    .select("Description").show(3, false)

show at cmd66.sc:7

1 / 1

show at cmd66.sc:7

1 / 1

show at cmd66.sc:7

1 / 1

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



simpleColors: Seq[String] = List("black", "white", "red", "green", "blue")
selectedColumns: Seq[Column] = List(
  contains(Description, BLACK) AS `is_black`,
  contains(Description, WHITE) AS `is_white`,
  contains(Description, RED) AS `is_red`,
  contains(Description, GREEN) AS `is_green`,
  contains(Description, BLUE) AS `is_blue`,
  unresolvedstar()
)

In [68]:
// inferSchema try to match timezone as possible
df.printSchema

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [69]:
val dateDF = spark.range(10)
    .withColumn("today", current_date())
    .withColumn("now", current_timestamp())
dateDF.createOrReplaceTempView("dataTable")
dateDF.printSchema

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



dateDF: DataFrame = [id: bigint, today: date ... 1 more field]

In [70]:
// date_sum, date_add
dateDF.select(date_sub($"today", 5), date_add($"today", 5)).show(3)

show at cmd69.sc:1

1 / 1

show at cmd69.sc:1

1 / 1

show at cmd69.sc:1

...

show at cmd69.sc:1

2 / 2

show at cmd69.sc:1

2 / 2

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2020-07-06|        2020-07-16|
|        2020-07-06|        2020-07-16|
|        2020-07-06|        2020-07-16|
+------------------+------------------+
only showing top 3 rows



In [71]:
spark.sql("""
    SELECT date_sub(today, 5), date_add(today, 5)
    FROM dataTable
""")show(3)

show at cmd70.sc:4

1 / 1

show at cmd70.sc:4

1 / 1

show at cmd70.sc:4

...

show at cmd70.sc:4

2 / 2

show at cmd70.sc:4

2 / 2

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2020-07-06|        2020-07-16|
|        2020-07-06|        2020-07-16|
|        2020-07-06|        2020-07-16|
+------------------+------------------+
only showing top 3 rows



In [74]:
spark.sql("""
    SELECT datediff(date_sub(today, 7), today) AS datediff, months_between('2016-01-01','2017-05-22') AS months_between
    FROM dataTable
""").show(1, false)

show at cmd73.sc:4

1 / 1

show at cmd73.sc:4

1 / 1

show at cmd73.sc:4

1 / 1

+--------+--------------+
|datediff|months_between|
+--------+--------------+
|-7      |-16.67741935  |
+--------+--------------+
only showing top 1 row



In [80]:
// SimpleDateFormat
spark.sql("""
    SELECT to_date(today, 'yyyy-dd-MM'), to_date('07-2020-11', 'MM-yyyy-dd'), to_date(today)
    FROM dateTable
""").show(1, false)

show at cmd79.sc:4

1 / 1

show at cmd79.sc:4

1 / 1

show at cmd79.sc:4

1 / 1

+----------------------------------------+-----------------------------------+--------------------------+
|to_date(datetable.`today`, 'yyyy-dd-MM')|to_date('07-2020-11', 'MM-yyyy-dd')|to_date(datetable.`today`)|
+----------------------------------------+-----------------------------------+--------------------------+
|2020-07-11                              |2020-07-11                         |2020-07-11                |
+----------------------------------------+-----------------------------------+--------------------------+
only showing top 1 row



In [82]:
spark.sql("""
    SELECT to_timestamp('20201107', 'yyyyMMdd'), to_timestamp('07-11-2020', 'MM-dd-yyyy')
    FROM datatable
""").show(1, false)

show at cmd81.sc:4

1 / 1

show at cmd81.sc:4

1 / 1

show at cmd81.sc:4

1 / 1

+------------------------------------+----------------------------------------+
|to_timestamp('20201107', 'yyyyMMdd')|to_timestamp('07-11-2020', 'MM-dd-yyyy')|
+------------------------------------+----------------------------------------+
|2020-11-07 00:00:00                 |2020-07-11 00:00:00                     |
+------------------------------------+----------------------------------------+
only showing top 1 row



In [84]:
spark.sql("""
    SELECT cast(to_date('2019-09-14', 'yyyy-MM-dd') AS timestamp)
""").show(1, false)

show at cmd83.sc:3

1 / 1

show at cmd83.sc:3

1 / 1

show at cmd83.sc:3

1 / 1

+------------------------------------------------------+
|CAST(to_date('2019-09-14', 'yyyy-MM-dd') AS TIMESTAMP)|
+------------------------------------------------------+
|2019-09-14 00:00:00                                   |
+------------------------------------------------------+



In [85]:
df.select(coalesce($"Description", $"CustomerId")).show()

show at cmd84.sc:1

1 / 1

show at cmd84.sc:1

1 / 1

show at cmd84.sc:1

1 / 1

+---------------------------------+
|coalesce(Description, CustomerId)|
+---------------------------------+
|             WHITE HANGING HEA...|
|              WHITE METAL LANTERN|
|             CREAM CUPID HEART...|
|             KNITTED UNION FLA...|
|             RED WOOLLY HOTTIE...|
|             SET 7 BABUSHKA NE...|
|             GLASS STAR FROSTE...|
|             HAND WARMER UNION...|
|             HAND WARMER RED P...|
|             ASSORTED COLOUR B...|
|             POPPY'S PLAYHOUSE...|
|             POPPY'S PLAYHOUSE...|
|             FELTCRAFT PRINCES...|
|             IVORY KNITTED MUG...|
|             BOX OF 6 ASSORTED...|
|             BOX OF VINTAGE JI...|
|             BOX OF VINTAGE AL...|
|             HOME BUILDING BLO...|
|             LOVE BUILDING BLO...|
|             RECIPE BOX WITH M...|
+---------------------------------+
only showing top 20 rows



In [88]:
spark.sql("""
    SELECT 
        ifnull(null, 'return_value'),
        nullif('value', 'value'),
        nvl(null, 'return_value'),
        nvl2('not_null', 'return_value', 'else_value')
    FROM dfTable
    LIMIT 1
""").show()

show at cmd87.sc:1

1 / 1

show at cmd87.sc:1

1 / 1

show at cmd87.sc:1

...

show at cmd87.sc:1

1 / 1

show at cmd87.sc:1

1 / 1

+----------------------------+------------------------+-------------------------+----------------------------------------------+
|ifnull(NULL, 'return_value')|nullif('value', 'value')|nvl(NULL, 'return_value')|nvl2('not_null', 'return_value', 'else_value')|
+----------------------------+------------------------+-------------------------+----------------------------------------------+
|                return_value|                    null|             return_value|                                  return_value|
+----------------------------+------------------------+-------------------------+----------------------------------------------+



In [91]:
df.count

count at cmd90.sc:1

1 / 1

count at cmd90.sc:1

1 / 1

count at cmd90.sc:1

...

count at cmd90.sc:1

1 / 1

count at cmd90.sc:1

1 / 1

res90: Long = 3108L

In [93]:
// drop null
df.na.drop().count

count at cmd92.sc:1

1 / 1

count at cmd92.sc:1

1 / 1

count at cmd92.sc:1

...

count at cmd92.sc:1

1 / 1

count at cmd92.sc:1

1 / 1

res92: Long = 1968L

In [95]:
spark.sql("""
    SELECT *
    FROM dfTable
    WHERE Description IS NOT NULL
""").count

count at cmd94.sc:1

1 / 1

count at cmd94.sc:1

1 / 1

count at cmd94.sc:1

...

count at cmd94.sc:1

1 / 1

count at cmd94.sc:1

1 / 1

res94: Long = 3098L

In [97]:
// if all of selected col props is null
df.na.drop("all", Seq("StockCode", "InvoiceNo")).count

count at cmd96.sc:1

1 / 1

count at cmd96.sc:1

1 / 1

count at cmd96.sc:1

...

count at cmd96.sc:1

1 / 1

count at cmd96.sc:1

1 / 1

res96: Long = 3108L

In [98]:
// df in df
val complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")

complexDF: DataFrame = [complex: struct<Description: string, InvoiceNo: string>]

In [100]:
complexDF.select("complex.*").show()

show at cmd99.sc:1

1 / 1

show at cmd99.sc:1

1 / 1

show at cmd99.sc:1

1 / 1

+--------------------+---------+
|         Description|InvoiceNo|
+--------------------+---------+
|WHITE HANGING HEA...|   536365|
| WHITE METAL LANTERN|   536365|
|CREAM CUPID HEART...|   536365|
|KNITTED UNION FLA...|   536365|
|RED WOOLLY HOTTIE...|   536365|
|SET 7 BABUSHKA NE...|   536365|
|GLASS STAR FROSTE...|   536365|
|HAND WARMER UNION...|   536366|
|HAND WARMER RED P...|   536366|
|ASSORTED COLOUR B...|   536367|
|POPPY'S PLAYHOUSE...|   536367|
|POPPY'S PLAYHOUSE...|   536367|
|FELTCRAFT PRINCES...|   536367|
|IVORY KNITTED MUG...|   536367|
|BOX OF 6 ASSORTED...|   536367|
|BOX OF VINTAGE JI...|   536367|
|BOX OF VINTAGE AL...|   536367|
|HOME BUILDING BLO...|   536367|
|LOVE BUILDING BLO...|   536367|
|RECIPE BOX WITH M...|   536367|
+--------------------+---------+
only showing top 20 rows



In [101]:
// array
df.select(split($"Description", " ")).show(2)

show at cmd100.sc:1

1 / 1

show at cmd100.sc:1

1 / 1

show at cmd100.sc:1

1 / 1

+-------------------------+
|split(Description,  , -1)|
+-------------------------+
|     [WHITE, HANGING, ...|
|     [WHITE, METAL, LA...|
+-------------------------+
only showing top 2 rows



In [102]:
spark.sql("""
    SELECT Description, InvoiceNo, exploded
    FROM (SELECT *, split(Description, " ") as splitted FROM dfTable)
    LATERAL VIEW explode(splitted) as exploded
""")

res101: DataFrame = [Description: string, InvoiceNo: string ... 1 more field]

In [103]:
df.withColumn("splitted", split($"Description", " "))
    .withColumn("exploded", explode($"splitted"))
    .select("Description", "InvoiceNo", "exploded")
    .show(2)

show at cmd102.sc:4

1 / 1

show at cmd102.sc:4

1 / 1

show at cmd102.sc:4

1 / 1

+--------------------+---------+--------+
|         Description|InvoiceNo|exploded|
+--------------------+---------+--------+
|WHITE HANGING HEA...|   536365|   WHITE|
|WHITE HANGING HEA...|   536365| HANGING|
+--------------------+---------+--------+
only showing top 2 rows

